# Building AI Apps with LaunchDarkly and LangChain

## Overview

This notebook example demonstrates how to run the "Building AI Apps with LaunchDarkly and LangChain" sample codes in Amazon Bedrock.

Credits to the original writer who posted the excellent content.

You can follow along with this notebook while referring to the original post.
* https://docs.launchdarkly.com/guides/infrastructure/langchain

## Install dependencies

In [1]:
!pip install -qU boto3 langchain langchain-aws launchdarkly-server-sdk python-dotenv

In [2]:
!pip show langchain launchdarkly-server-sdk

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: jupyter_ai_magics, langchain-experimental, langserve, llama-index-llms-langchain
---
Name: launchdarkly-server-sdk
Version: 9.4.0
Summary: LaunchDarkly SDK for Python
Home-page: https://docs.launchdarkly.com/sdk/server-side/python
Author: LaunchDarkly
Author-email: dev@launchdarkly.com
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: certifi, expiringdict, launchdarkly-eventsource, pyRFC3339, semver, urllib3
Required-by: 


In [3]:
from dotenv import load_dotenv

# Load LaunchDarkly SDK from .env.
load_dotenv()

True

In [4]:
import os

LD_TEST_SDK = os.environ.get("LD-TEST-SDK")

## Setting up Amazon Bedrock with LangChain 🦜️🔗

In [5]:
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

## Writing a LangChain chain

In [6]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

helpful_prompt_template = PromptTemplate.from_template("""\
You are a helpful assistant. Please answer the user's question in a concise
and easy to understand manner.

Here is a question: {question}""")

llm = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

output_parser = StrOutputParser()
helpful_chain = helpful_prompt_template | llm | output_parser

In [7]:
question = "How can you use feature management to safely and quickly deploy software?"

In [8]:
# Stream
for chunk in helpful_chain.stream({"question": question}):
    print(chunk, end="", flush=True)

Feature management allows you to safely and quickly deploy software by:

1. Separating feature releases from code deployments: This allows you to test and validate new features independently without impacting the entire application.

2. Enabling feature flags/toggles: Feature flags let you turn features on/off remotely, making it easy to enable or disable functionality as needed.

3. Gradual rollouts: You can gradually roll out new features to a subset of users, monitor performance, and expand the rollout incrementally.

4. A/B testing: Feature management supports A/B testing, allowing you to compare different versions of a feature and determine the optimal configuration.

5. Reducing risk: By decoupling feature releases from deployments, you can reduce the risk of breaking changes and quickly roll back problematic features.

In summary, feature management provides the flexibility and control to deploy software safely and quickly by separating concerns, enabling remote feature toggles,

In [9]:
eliza_prompt_template = PromptTemplate.from_template("""\
You are a Rogerian therapist. Please answer the user's question with a question
on the same topic. If you don't understand the user's question, please ask them
to tell you more.

Here is a question: {question}""")
chain = eliza_prompt_template | llm | output_parser
chain.invoke({"question": question})

"That's an interesting question about feature management and software deployment. Could you tell me a bit more about your specific situation and what you're hoping to achieve with feature management? What are some of the challenges or concerns you've encountered in this area that you're hoping to address?"

## Setting up the LaunchDarkly SDK

In [10]:
import ldclient
from ldclient.config import Config

# Test Environment
ldclient.set_config(Config(LD_TEST_SDK))
client = ldclient.get()

## Selecting a prompt with a feature flag

In [11]:
from langchain.schema.runnable import ConfigurableField

prompt = helpful_prompt_template.configurable_alternatives(
        ConfigurableField(id="prompt"),
        default_key="helpful",
        eliza=eliza_prompt_template
)

chain = prompt | llm | output_parser

In [12]:
# Stream
for chunk in (chain.with_config(configurable={"prompt": "eliza"}).stream({"question": question})):
    print(chunk, end="", flush=True)

That's an interesting question about feature management and software deployment. Could you tell me a bit more about your specific situation and what you're hoping to achieve with feature management? What are some of the challenges or concerns you've encountered in this area that you're hoping to address?

In [13]:
from ldclient import Context

# Test Environment
context = Context.builder(LD_TEST_SDK).name("David").build()

prompt_variation = client.variation("langchain-prompts", context, "helpful")

In [14]:
prompt_variation

'helpful'

In [15]:
# Stream
for chunk in (chain.with_config(configurable={"prompt": prompt_variation}).stream({"question": question})):
    print(chunk, end="", flush=True)

Feature management allows you to safely and quickly deploy software by:

1. Separating feature deployment from code deployment: This allows you to release new features gradually and independently of the underlying codebase.

2. Enabling feature flags: Feature flags let you turn features on/off remotely without redeploying the entire application.

3. Controlling feature rollout: You can gradually roll out new features to a subset of users, monitor their performance, and expand the rollout as needed.

4. Providing a kill switch: If an issue is detected, you can quickly disable a problematic feature without impacting the entire application.

5. Gathering user feedback: Feature management allows you to collect data on feature usage and user behavior, informing future development decisions.

In summary, feature management gives you more control, flexibility, and visibility when deploying new software, leading to safer and faster releases.

# Adding a new variation to a feature flag

In [16]:
from langchain.schema.runnable import ConfigurableField

expert_prompt_template = PromptTemplate.from_template("""
You are an expert assistant. Reply to the the user's question with a detailed and technical answer.

Here is a question: {question}""")

prompt = helpful_prompt_template.configurable_alternatives(
        ConfigurableField(id="prompt"),
        default_key="helpful",
        eliza=eliza_prompt_template,
        # --- New alternative
        expert=expert_prompt_template
        # ---
)

chain = prompt | llm | output_parser

In [17]:
from ldclient import Context

context = Context.builder("sdk-a23ff921-ce1e-4066-90a9-b7b7d4945a34").name("David").build()
prompt_variation = client.variation("langchain-prompts", context, "helpful")

In [18]:
prompt_variation

'helpful'

In [19]:
# Stream
for chunk in (chain.with_config(configurable={"prompt": prompt_variation}).stream({"question": question})):
    print(chunk, end="", flush=True)

Feature management allows you to safely and quickly deploy software by:

1. Enabling Incremental Rollouts: You can gradually roll out new features to a subset of users, monitor their performance, and then expand the rollout.

2. Providing Feature Flags: Feature flags allow you to enable or disable specific features remotely, without the need for a full code deployment.

3. Reducing Risk: By using feature management, you can reduce the risk of deploying new features by isolating them and controlling their release.

4. Gathering Feedback: You can gather user feedback on new features before a full release, allowing you to make adjustments based on real-world usage.

5. Improving Agility: Feature management enables faster development cycles and more frequent, smaller deployments, improving your overall agility.

In summary, feature management helps you deploy software more safely and quickly by allowing you to control feature releases, gather feedback, and reduce the risk of deployments.

# Storing prompts in feature flags variations

In [20]:
prompt = helpful_prompt_template.configurable_fields(
    template=ConfigurableField(
        id="prompt_template",
    )
)

chain = prompt | llm | output_parser

variation = client.variation("langchain-prompt-templates", context, prompt.default.template)

In [21]:
variation

{'template': "You are a Rogerian therapist. Please answer the user's question with a question on the same topic. If you don't understand the user's question, please ask them to tell you more.\n\nHere is a question: {question}"}

In [22]:
# Stream
for chunk in (chain.with_config(configurable={"prompt_template": variation.get("template")}).stream({"question": question})):
    print(chunk, end="", flush=True)

That's an interesting question about feature management and software deployment. Could you tell me a bit more about your specific goals or challenges in this area? What are you hoping to achieve with feature management, and what concerns do you have around safely and quickly deploying your software? I'd be happy to explore this topic further with you, but I want to make sure I understand your perspective and needs first.

# Storing prompt template variables in feature flags

In [23]:
prompt = PromptTemplate.from_template("""\
You are an {adjective} assistant. Answer the user's question {instruction}. Question: {question}""")
chain = prompt | llm | output_parser

In [24]:
variation = client.variation("langchain-template-variables", context,
   {"adjective": "helpful", "instruction": "in a concise and easy to understand manner."})

In [25]:
variation

{'adjective': 'sympathetic',
 'instruction': 'by asking them to tell you more about their question.'}

In [26]:
# Stream
for chunk in chain.stream({"question": question, **variation}):
    print(chunk, end="", flush=True)

I'd be happy to help you with that! Could you tell me a bit more about the context of your question? What kind of software are you looking to deploy, and what are your main goals or concerns when it comes to the deployment process? The more details you can provide, the better I can tailor my response to your specific needs.

# Selecting and configuring LLMs

In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage

llm = llm.configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="claude",
    titan=ChatBedrock(model_id="anthropic.titan-instant-v1"),
    command=ChatBedrock(model_id="anthropic.titan-instant-v1"),
    mistral=ChatBedrock(model_id="mistral.mistral-7b-instruct-v0:2"),
)

chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant. Please answer the user's question in a concise and easy to understand manner."),
    ("human", "{question}")
])

chat_chain = chat_prompt_template | llm | output_parser

In [28]:
variation = client.variation("langchain-llm-models", context, "claude")
variation

'mistral'

In [29]:
# Stream
for chunk in (chat_chain.with_config(configurable={"model": variation}).stream({"question": question})):
    print(chunk, end="", flush=True)

 Feature management is a practice that allows you to control the release of new features or changes to your software in a safe and controlled manner. Here's how you can use it to deploy software quickly and safely:

1. **Isolate new features:** Use feature flags to isolate new features or changes from the rest of the codebase. This allows you to test the new feature in a controlled environment without affecting the entire user base.

2. **Gradually roll out features:** Use a phased rollout strategy to gradually release new features to a small percentage of users. This helps you identify and fix any issues before releasing the feature to a larger audience.

3. **Monitor feature performance:** Use analytics and monitoring tools to track the performance of new features in real-time. This allows you to quickly identify any issues and make adjustments as needed.

4. **Roll back features:** If a new feature causes issues or negatively impacts user experience, you can use feature flags to rol

### Configure the parameters of a model

In [30]:
llm = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0").configurable_fields(
    model_id=ConfigurableField(id="model_id"),
    model_kwargs=ConfigurableField(id="model_kwargs")
)
chain = helpful_prompt_template | llm | output_parser

In [31]:
variation = client.variation("langchain-llm-model-kwargs", context,
    {"model_id": "anthropic.claude-3-haiku-20240307-v1:0", "model_kwargs": {"temperature": 0.5}})

In [32]:
variation

{'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}

In [33]:
# Stream
for chunk in (chat_chain.with_config(configurable={**variation}).stream({"question": question})):
    print(chunk, end="", flush=True)

Feature management is a software development practice that allows you to safely and quickly deploy software by controlling the release of new features. Here are some key ways feature management can help:

1. Gradual Rollout: Feature management enables you to gradually roll out new features to a subset of users first, rather than deploying to all users at once. This allows you to test the feature, gather feedback, and make adjustments before a full release.

2. Feature Flags: Feature flags are toggles that allow you to turn features on or off remotely, without having to redeploy the entire application. This makes it easy to quickly enable, disable, or A/B test new features.

3. Targeted Releases: You can use feature management to target the release of new features to specific user segments, regions, or devices. This helps minimize the impact of issues and allows you to test features with a limited audience first.

4. Reduced Risk: By controlling the release of new features, feature mana